In [2]:
import numpy as np
import pandas as pd

In [4]:
df1=pd.read_csv("../r2Data/user_vendor_series_data.csv")

## Buliding basic collaborative model

In [15]:
df1.drop(columns=['address','useful','funny','cool'],inplace=True)

In [16]:
df1.head(3)

,user_id,business_id,stars_x,text,name,city,latitude,longitude,review_count,is_open,categories
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,North Wales,40.210196,-75.223639,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,Philadelphia,39.952103,-75.172753,144,0,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,Tucson,32.207233,-110.980864,47,1,"Restaurants, Breakfast & Brunch"


In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 599959 entries, 0 to 599999
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       599959 non-null  object 
 1   business_id   599959 non-null  object 
 2   stars_x       599959 non-null  int64  
 3   text          599959 non-null  object 
 4   name          599959 non-null  object 
 5   city          599959 non-null  object 
 6   latitude      599959 non-null  float64
 7   longitude     599959 non-null  float64
 8   review_count  599959 non-null  int64  
 9   is_open       599959 non-null  int64  
 10  categories    599959 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 54.9+ MB


In [24]:
pd.pivot_table(df1,index=['business_id'],columns=['user_id'],values='stars_x')

C:\Users\ujwal\AppData\Local\Temp\ipykernel_17788\813482887.py:1: PerformanceWarning: The following operation may generate 5585081268 cells in the resulting pandas object.
  pd.pivot_table(df1,index=['business_id'],columns=['user_id'],values='stars_x')


MemoryError: Unable to allocate 41.6 GiB for an array with shape (14989, 372612) and data type float64